# Demonstrate Seq2Seq Wrapper with twitter chat log

In [36]:
import tensorflow as tf
import numpy as np

# preprocessed data
from datasets.twitter import data
import data_utils

In [37]:
# load data from pickle and npy files
metadata, idx_q, idx_a = data.load_data(PATH='datasets/cornell_corpus/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

In [38]:
# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 16
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 512

In [39]:
import seq2seq_wrapper

In [40]:
import importlib
importlib.reload(seq2seq_wrapper)

<module 'seq2seq_wrapper' from 'C:\\Users\\ricsi\\Documents\\GitHub\\Python-projects\\seq2seq_chatbot_projects\\practical_seq2seq_untouched\\seq2seq_wrapper.py'>

In [41]:
model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
                               yseq_len=yseq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/cornell_corpus/',
                               emb_dim=emb_dim,
                               num_layers=3
                               )

<log> Building Graph </log>

In [42]:
val_batch_gen = data_utils.rand_batch_gen(validX, validY, 256)
test_batch_gen = data_utils.rand_batch_gen(testX, testY, 256)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)

In [9]:
sess = model.train(train_batch_gen, val_batch_gen)


<log> Training started </log>

Model saved to disk at iteration #500
val   loss : 3.370399
Interrupted by user at iteration 565


In [43]:
sess = model.restore_last_session()

In [13]:
input_ = test_batch_gen.__next__()[0]
output = model.predict(sess, input_)
print(output.shape)

(256, 20)


In [14]:
replies = []
for ii, oi in zip(input_.T, output):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = data_utils.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    if decoded.count('unk') == 0:
        if decoded not in replies:
            print('q : [{0}]; a : [{1}]'.format(q, ' '.join(decoded)))
            replies.append(decoded)

q : [im sure you can do better than this little buddy]; a : [he was not a joke but he was a pathological liar and a lot of people]
q : [im crying angry tears and thats not good for anyone in my energy field]; a : [or the time ]
q : [headphone land]; a : [the whole season is a good place to get a new yorker]
q : [hilarious to watch the panel desperately tap dance around the fact that the debate hands down as says sad]; a : [i think the same thing is the worst]
q : [happy birthday dude  fuck it up at your new job lol it was fun working with you ]; a : [thanks bro i miss you too ]
q : [its unfortunate that im so young and i already feel that dating in this generation is a nightmare]; a : [the whole thing is that the reason is a joke]
q : [limited parking at park entrance on unk road can use american unk parking lot and walk down to entrance]; a : [we will be in the same side]
q : [oh god not the unk]; a : [i know i was just thinking about this]
q : [trailer park boys is not what i was thi

In [44]:
import tensorflow as tf
import numpy as np
import random

# preprocessed data
from datasets.cornell_corpus import data
import data_utils

# load data from pickle and npy files
metadata, idx_q, idx_a = data.load_data(PATH='datasets/cornell_corpus/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

temp=0.3

input_ = test_batch_gen.__next__()[0]
output = model.predict(sess, input_)
output_max=np.argmax(output,axis=2)
print(output_max[0,:])

pred=np.zeros((256,20),dtype=np.int32)
for inst in range(256):
    for word in range(20):
        #output[0,word,output[0,word,:]<0]=0
        #output[0,word,:]+=np.abs(np.amin(output[0,word,:]))
        sum_all=np.sum(np.exp(output[inst,word,:]/temp))
        output[inst,word,:]=np.exp(output[inst,word,:]/temp)/sum_all
    
        r=random.uniform(0,1)
        s=0
        broke=False
        for i in range(8002):
            s+=output[inst,word,i]
            if s>=r:
                pred[inst,word]=i
                broke =True
                break
        if not broke:
            pred[inst,word]=8001

output=pred
print(pred[0,:])

replies = []
for ii, oi_max,oi_pred in zip(input_.T, output_max,pred):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded_max = data_utils.decode(sequence=oi_max, lookup=metadata['idx2w'], separator=' ').split(' ')
    decoded_pred = data_utils.decode(sequence=oi_pred, lookup=metadata['idx2w'], separator=' ').split(' ')
    #if decoded.count('unk') == 0:
        #if decoded not in replies:
    print('q : [{0}]; max a : [{1}]'.format(q, ' '.join(decoded_max)))
    print('q : [{0}]; pred a : [{1}]'.format(q, ' '.join(decoded_pred)))
    replies.append(decoded_max)

[32 31  6  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
[ 32  31   6 115   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
q : [yes indeed he has been a unk case now that uyouu have him in unk have him havent you]; max a : [he was a unk]
q : [yes indeed he has been a unk case now that uyouu have him in unk have him havent you]; pred a : [he was a little]
q : [unk a spare]; max a : [i dont know what you mean]
q : [unk a spare]; pred a : [you dont know you mean]
q : [go on]; max a : [i dont know]
q : [go on]; pred a : [unk dont know]
q : [oh my god sidney you ran fast or what hey]; max a : [i dont know]
q : [oh my god sidney you ran fast or what hey]; pred a : [i dont know]
q : [is the day so young]; max a : [i dont know]
q : [is the day so young]; pred a : [you know know]
q : [know where he is]; max a : [no i dont know]
q : [know where he is]; pred a : [no i dont know]
q : [you know were going to give this party for you whether you like it or not]; m